## DEEP_CONVULTIONAL_NET_FOR_DEVANAGARI_CHARACTER_CLASSIFICATION

#### IMPORTING_DEPENDENCIES

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.callbacks import TensorBoard

Using TensorFlow backend.


#### LOADING_DATA

In [2]:
datadir_train = 'C://Users//Siddharth//Desktop//AI//devanagari-character-dataset-large//dhcd//train'
datadir_test = 'C://Users//Siddharth//Desktop//AI//devanagari-character-dataset-large//dhcd//test'
categories = []
for i in range(45):
    categories.append(str(i))

In [3]:
data_train = []
data_test = []
def create_train_data():                            #create train and test data 
    for category in categories:
        path_train = os.path.join(datadir_train,category)
        class_num = categories.index(category)
        for img_train in os.listdir(path_train):
            img_array_train = cv2.imread(os.path.join(path_train,img_train), cv2.IMREAD_GRAYSCALE)
            data_train.append([img_array_train, class_num])

def create_test_data():
    for category in categories:
        path_test = os.path.join(datadir_test,category)
        class_num = categories.index(category)
        for img_test in os.listdir(path_test):
            img_array_test = cv2.imread(os.path.join(path_test,img_test), cv2.IMREAD_GRAYSCALE)
            data_test.append([img_array_test, class_num])            
            
create_train_data()  
create_test_data()

In [4]:
print(len(data_train))


76500


In [5]:
print(len(data_test))

13500


In [6]:
import random #TO SHUFFLE THE TRAIN AND TEST DATA
random.shuffle(data_train)
random.shuffle(data_test)

In [7]:
X_train = []              #initializing lists of train and test data
X_test = []
y_train = []
y_test = []

In [8]:
for features_train, label_train in data_train:   #CREATING A LIST OF TRAIN AND TEST DATA
    X_train.append(features_train)
    y_train.append(label_train)
for features_test, label_test in data_test:
    X_test.append(features_test)
    y_test.append(label_test)    

X_train = np.array(X_train)    #CONVERTING THE LIST TO NUMPY ARRAY
y_train = np.array(y_train)
X_test = np.array(X_test)    
y_test = np.array(y_test)

In [9]:
import pickle                           #SAVING AND LOADING THE DATA USING PICKLE MODULE
pickle_out = open('X_train.pickle','wb')
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open('y_train.pickle','wb')
pickle.dump(y_train, pickle_out)
pickle_out.close()

pickle_out = open('X_test.pickle','wb')
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open('y_test.pickle','wb')
pickle.dump(y_test, pickle_out)
pickle_out.close()

In [10]:
pickle_in = open('X_train.pickle','rb')
X_train = pickle.load(pickle_in)

pickle_in = open('y_train.pickle','rb')
y_train = pickle.load(pickle_in)

pickle_in = open('X_test.pickle','rb')
X_test = pickle.load(pickle_in)

pickle_in = open('y_test.pickle','rb')
y_test = pickle.load(pickle_in)

In [11]:
print(X_train.shape) 

(76500, 32, 32)


In [12]:
print(y_train.shape)

(76500,)


#### PREPROCESSING

In [13]:
X_train = X_train.reshape(76500,32,32,1).astype('float32')   #RESHAPING TRAIN AND TEST DATA
X_test = X_test.reshape(13500,32,32,1).astype('float32')

In [14]:
X_train /= 255                                               #NORMALIZE X_train AND X_train
X_test /= 255

In [15]:
n_classes = 46                                               #DEFINING NUMBER OF CLASSES
y_train = keras.utils.to_categorical(y_train,n_classes)
y_test = keras.utils.to_categorical(y_test,n_classes)

In [16]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,        #create a instance of Tensorboard
                          write_graph=True, write_images=False)

#### CREATE_SEQUENTIAL_MODEL(CNN)

In [17]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape = (32,32,1))) #1st convulutional layer
model.add(MaxPooling2D(pool_size = (2,2)))                                             #max pooling layer
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                        #2nd convulutional layer
model.add(MaxPooling2D(pool_size = (2,2)))                                             #max polling layer
model.add(BatchNormalization())                                                        #batch normalisation layer
model.add(Flatten())                                                                   #flattening layer
model.add(Dense(128, activation = 'relu'))                                             #1st dense layer
model.add(Dropout(0.25))                                                               #dropout layer to avoid overfitting
model.add(Dense(n_classes, activation = 'softmax'))                                    #output layer

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               295040    
__________

In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) #COMPILING THE MODEL

In [20]:
model.fit(X_train, y_train, batch_size = 32, epochs = 20, verbose = 1, validation_data = [X_test, y_test], callbacks=[tensorboard])

Train on 76500 samples, validate on 13500 samples
Epoch 1/20
76500/76500 [==============================] - 51s 661us/step - loss: 0.4461 - acc: 0.8703 - val_loss: 0.1336 - val_acc: 0.9577
Epoch 2/20
76500/76500 [==============================] - 50s 659us/step - loss: 0.1546 - acc: 0.9525 - val_loss: 0.1421 - val_acc: 0.9599
Epoch 3/20
76500/76500 [==============================] - 50s 657us/step - loss: 0.1098 - acc: 0.9660 - val_loss: 0.0958 - val_acc: 0.9736
Epoch 4/20
76500/76500 [==============================] - 50s 658us/step - loss: 0.0870 - acc: 0.9727 - val_loss: 0.1046 - val_acc: 0.97180s - loss: 0.0870 - acc: 
Epoch 5/20
76500/76500 [==============================] - 50s 658us/step - loss: 0.0730 - acc: 0.9768 - val_loss: 0.0974 - val_acc: 0.9733
Epoch 6/20
76500/76500 [==============================] - 50s 658us/step - loss: 0.0633 - acc: 0.9797 - val_loss: 0.0792 - val_acc: 0.9807
Epoch 7/20
76500/76500 [==============================] - 50s 656us/step - loss: 0.0567 - a